In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
plt.rcParams['figure.figsize'] = (10, 8)
# Any results you write to the current directory are saved as output.

In [40]:
df = pd.read_csv('epldata_final.csv')

In [41]:
df.head()

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2.0,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1.0,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2.0,France,0,4,1,1,0


In [42]:
df['region'].fillna((df['region'].median()), inplace=True)
#df.isnull().any()

In [43]:
def create_df(data_fram, feature_list):
    out = data_fram
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

df1 = df
features = ['club', 'position', 'nationality', 'position_cat', 'region']
for feature in features:
    df1[feature] = df1[feature].astype('object')
df1 = create_df(df1, features)
df1.head()

,name,age,market_value,page_views,fpl_value,fpl_sel,fpl_points,new_foreign,age_cat,club_id,...,nationality_Venezuela,nationality_Wales,position_cat_1,position_cat_2,position_cat_3,position_cat_4,region_1.0,region_2.0,region_3.0,region_4.0
0,Alexis Sanchez,28,65.0,4329,12.0,17.10%,264,0,4,1,...,0,0,1,0,0,0,0,0,1,0
1,Mesut Ozil,28,50.0,4395,9.5,5.60%,167,0,4,1,...,0,0,1,0,0,0,0,1,0,0
2,Petr Cech,35,7.0,1529,5.5,5.90%,134,0,6,1,...,0,0,0,0,0,1,0,1,0,0
3,Theo Walcott,28,20.0,2393,7.5,1.50%,122,0,4,1,...,0,0,1,0,0,0,1,0,0,0
4,Laurent Koscielny,31,22.0,912,6.0,0.70%,121,0,4,1,...,0,0,0,0,1,0,0,1,0,0


In [44]:
y = df1['market_value']
df1 = df1.drop(['name', 'market_value', 'fpl_sel'], axis=1)

In [45]:
from sklearn import preprocessing
# normalize the data attributes
normalized_X = preprocessing.normalize(df1)
# standardize the data attributes
standardized_X = preprocessing.scale(df1)

In [46]:
X_train, X_valid, y_train, y_valid = train_test_split(df1, y, test_size = 0.3, random_state=17)

In [47]:
clf_tree = DecisionTreeRegressor(max_depth=4, random_state=17)
clf_tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=17, splitter='best')

In [48]:
predicted = clf_tree.predict(X_valid)
mae = mean_absolute_error(y_valid, predicted)
mae

5.602330442709841

In [49]:
dict_model_result = {}

# bagging over trees
br = BaggingRegressor(base_estimator=clf_tree, n_estimators=40, random_state=17)
br.fit(X_train, y_train)
br_preds = br.predict(X_valid)
dict_model_result['Accuracy Bagging on valid set'] = mean_absolute_error(y_valid, br_preds)

In [50]:
mae_scorer = make_scorer(mean_absolute_error)
params = {'n_estimators': [40, 42], 'base_estimator__max_leaf_nodes':[10, 15], 'base_estimator__max_depth':[4, 5, 6]}

# Grid Search to determine best parameters
br_grid = GridSearchCV(estimator=br, param_grid=params, scoring=mae_scorer, cv=5, n_jobs=-1)
br_grid.fit(X_train, y_train)
best_params = br_grid.best_params_
print('лучшее значение скора на кросс-валидации -', br_grid.best_score_)
print('лучшие характеристики -',best_params)

лучшее значение скора на кросс-валидации - 4.316818385007596
лучшие характеристики - {'base_estimator__max_depth': 4, 'base_estimator__max_leaf_nodes': 10, 'n_estimators': 40}


In [51]:
rfr = RandomForestRegressor(max_depth=4,  max_features=None, random_state=17)
rfr.fit(X_train, y_train)
rfr_preds = rfr.predict(X_valid)
dict_model_result['Accuracy Random Forest on valid set'] = mean_absolute_error(y_valid, rfr_preds)

In [52]:
max_depth_values = range(4, 8)
forest_params = {'max_depth': max_depth_values}

random_forest_regressor = RandomForestRegressor(random_state = 17)
random_forest_regressor_grid = GridSearchCV(random_forest_regressor, param_grid=forest_params, scoring=mae_scorer, cv = 5)
random_forest_regressor_grid.fit(X_train, y_train)
print('лучшее значение скора на кросс-валидации -', random_forest_regressor_grid.best_score_)

лучшее значение скора на кросс-валидации - 4.274021117033404


In [53]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_predict = lr.predict(X_valid)
dict_model_result['Accuracy Logistic Regression on valid set'] = mean_absolute_error(y_valid, lr_predict)

In [54]:
knn = KNeighborsRegressor(n_neighbors=6)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_valid)
dict_model_result['Accuracy knn on valid set'] = mean_absolute_error(y_valid, knn_pred)

In [55]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsRegressor(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(3, 15)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
knn_grid.best_params_, knn_grid.best_score_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    8.2s finished


({'knn__n_neighbors': 10}, 0.5219316310953624)

In [56]:
def Simple_Ensembles(*args):
    Ensembles_predict = []
    arr = np.array(args).T
    for i in range(0, arr.shape[0]):
        one_line = list(arr[i])
        set_one_line = set(one_line)
        
        most_common = None
        qty_most_common = 0

        for item in set_one_line:
            qty = one_line.count(item)
            if qty > qty_most_common:
                qty_most_common = qty
                most_common = item

        Ensembles_predict.append(most_common)
    return Ensembles_predict     

In [57]:
result = Simple_Ensembles(br_preds, rfr_preds, lr_predict, knn_pred)

In [58]:
result = np.array([result])
result = result.reshape((139,))

In [59]:
mae = mean_absolute_error(y_valid, result)
mae

4.760999238410592

In [60]:
X_train_part1 = X_train.reset_index(drop=True).loc[0:220,]
X_train_part2 = X_train.reset_index(drop=True).loc[221:,]

y_train_part1 = y_train.reset_index(drop=True).loc[0:220,]
y_train_part2 = y_train.reset_index(drop=True).loc[221:,]

In [61]:
# bagging over trees
bc = BaggingRegressor(base_estimator=clf_tree, n_estimators=40, random_state=17)
bc.fit(X_train_part1, y_train_part1)
bc_preds_part2 = bc.predict(X_train_part2)
mean_absolute_error(y_train_part2, bc_preds_part2)                                            

4.051736890170098

In [62]:
bc_valid = bc.predict(X_valid)
mean_absolute_error(y_valid, bc_valid) 

4.249953965783229

In [63]:
knn = KNeighborsRegressor(n_neighbors=6)
knn.fit(X_train_part1, y_train_part1)
knn_pred = knn.predict(X_train_part2)
mean_absolute_error(y_train_part2, knn_pred)

5.752887788778876

In [64]:
knn_valid = knn.predict(X_valid)
mean_absolute_error(y_valid, knn_valid) 

4.7544364508393295

In [65]:
logit = LinearRegression()
logit.fit(X_train_part1, y_train_part1)
log_predict = logit.predict(X_train_part2)
mean_absolute_error(y_train_part2, log_predict)

5.011259005836436

In [66]:
log_predict_valid = logit.predict(X_valid)
mean_absolute_error(y_valid, log_predict_valid) 

5.067237558228073

In [67]:
rfc = RandomForestRegressor(max_depth=4,  max_features=None, random_state=17)
rfc.fit(X_train_part1, y_train_part1)
rfc_preds_part2 = rfc.predict(X_train_part2)
mean_absolute_error(y_train_part2, rfc_preds_part2)

3.9021057925481974

In [68]:
rfc_valid = rfc.predict(X_valid)
mean_absolute_error(y_valid, rfc_valid)

4.109696857517745

In [69]:
d1 = {'BaggingRegressor_train': bc_preds_part2, 'RandomForestRegressor_train': rfc_preds_part2, \
      'KNeighborsRegressor_train': knn_pred, 'LinearRegressor_train': log_predict}
df1 = pd.DataFrame(data=d1)

d2 = {'BaggingRegressor_test': bc_valid, 'RandomForestRegressor_test': rfc_valid, \
      'KNeighborsRegressor_test': knn_valid, 'LinearRegressor_test': log_predict_valid}
df2 = pd.DataFrame(data=d2)

In [70]:
lr = LinearRegression()
lr.fit(df1, y_train_part2)
lr_predict = lr.predict(df2)
mean_absolute_error(y_valid, lr_predict)

4.43028766875916

In [71]:
mean_squared_error(y_valid, lr_predict)

44.12600633299211

In [72]:
r2_score(y_valid, lr_predict)

0.6179034253858511